In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession 
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Path to the dataset
path = 'dataset/train/'
class_names = sorted(os.listdir(path))
num_classes = len(class_names)

# Image size
img_size = (192, 192)  # ResNet50 input size

# Load and preprocess data
labels = []
images = []

for cl in class_names:
    for img in os.listdir(path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        labels.append(label)
        
        image = cv2.imread(path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        images.append(image)

# Convert to numpy arrays
labels = np.asarray(labels)
images = np.asarray(images)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.15, random_state=42)

# Load ResNet50 pre-trained on ImageNet without top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=img_size + (3,))

# Add custom top layers for skin diseases classification
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint('resnet50_model/model{epoch:02d}')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 192, 192, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 198, 198, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 96, 96, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 96, 96, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [6]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[checkpoint_callback])

Epoch 1/20


179/179 [==============================] - ETA: 0s - loss: 1.6825 - accuracy: 0.4195INFO:tensorflow:Assets written to: resnet50_model\model01\assets


INFO:tensorflow:Assets written to: resnet50_model\model01\assets


179/179 [==============================] - 979s 4s/step - loss: 1.6825 - accuracy: 0.4195 - val_loss: 1.5447 - val_accuracy: 0.4738
Epoch 2/20
179/179 [==============================] - ETA: 0s - loss: 1.3155 - accuracy: 0.5402INFO:tensorflow:Assets written to: resnet50_model\model02\assets


INFO:tensorflow:Assets written to: resnet50_model\model02\assets


179/179 [==============================] - 759s 4s/step - loss: 1.3155 - accuracy: 0.5402 - val_loss: 1.4813 - val_accuracy: 0.4946
Epoch 3/20
179/179 [==============================] - ETA: 0s - loss: 1.1172 - accuracy: 0.6125INFO:tensorflow:Assets written to: resnet50_model\model03\assets


...

179/179 [==============================] - 620s 3s/step - loss: 0.1048 - accuracy: 0.9658 - val_loss: 2.4599 - val_accuracy: 0.5549
Epoch 19/20
179/179 [==============================] - ETA: 0s - loss: 0.0834 - accuracy: 0.9750INFO:tensorflow:Assets written to: resnet50_model\model19\assets


INFO:tensorflow:Assets written to: resnet50_model\model19\assets


179/179 [==============================] - 483s 3s/step - loss: 0.0834 - accuracy: 0.9750 - val_loss: 2.5100 - val_accuracy: 0.5717
Epoch 20/20
179/179 [==============================] - ETA: 0s - loss: 0.0258 - accuracy: 0.9948INFO:tensorflow:Assets written to: resnet50_model\model20\assets


INFO:tensorflow:Assets written to: resnet50_model\model20\assets


179/179 [==============================] - 446s 2s/step - loss: 0.0258 - accuracy: 0.9948 - val_loss: 2.4496 - val_accuracy: 0.5865


In [13]:
test_path = 'dataset/test/'

# Load and preprocess test data
test_labels = []
test_images = []

class_names = sorted(os.listdir(test_path))
num_classes = len(class_names)

for cl in class_names:
    for img in os.listdir(test_path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        test_labels.append(label)
        
        image = cv2.imread(test_path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        test_images.append(image)

# Convert to numpy arrays
test_labels = np.asarray(test_labels)
test_images = np.asarray(test_images)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

56/56 [==============================] - 128s 2s/step - loss: 1.7971 - accuracy: 0.4165
Test Loss: 1.7971
Test Accuracy: 0.4165


In [8]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# Path to the dataset
path = 'dataset/train/'
class_names = sorted(os.listdir(path))
num_classes = len(class_names)

# Image size
img_size = (192, 192)  # ResNet50 input size

# Load and preprocess data
labels = []
images = []

for cl in class_names:
    for img in os.listdir(path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        labels.append(label)
        
        image = cv2.imread(path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        images.append(image)

# Convert to numpy arrays
labels = np.asarray(labels)
images = np.asarray(images)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.15, random_state=42)

# Load ResNet50 pre-trained on ImageNet without top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=img_size + (3,))

# Add custom top layers for skin diseases classification
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout layer
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout layer
output = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint('resnet50_model/model{epoch:02d}')

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 192, 192, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 198, 198, 3)          0         ['input_3[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 96, 96, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 96, 96, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [9]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[checkpoint_callback])

Epoch 1/20


179/179 [==============================] - ETA: 0s - loss: 2.0825 - accuracy: 0.2775

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x00000206468A2C50>>
Traceback (most recent call last):
  File "c:\Users\surya\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
test_path = 'dataset/test/'

# Load and preprocess test data
test_labels = []
test_images = []

class_names = sorted(os.listdir(test_path))
num_classes = len(class_names)

for cl in class_names:
    for img in os.listdir(test_path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        test_labels.append(label)
        
        image = cv2.imread(test_path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        test_images.append(image)

# Convert to numpy arrays
test_labels = np.asarray(test_labels)
test_images = np.asarray(test_images)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

5/5 [==============================] - 8s 1s/step - loss: 1.4766 - accuracy: 0.4645
Test Loss: 1.4766
Test Accuracy: 0.4645
